In [1]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as offline
import pandas as pd 
import numpy as np
import plotly.io as pio
from scipy import stats 
offline.init_notebook_mode(connected=True)
from IPython.display import display



In [ ]:
arr=!echo $LOGDIR
FOLDER=arr[0]

In [3]:
#Place / at the end of path
FOLDER="/zfs/dicelab/DigitalPlant/results/June_04_08_15_24_node1709/connection_strings_tier1.list_run/"
# pd.set_option('display.max_columns', None)


In [2]:
#FOLDER="./"
df = pd.read_csv(FOLDER+'trial0/all.csv', na_filter= False)

# devices = df.groupby('deviceNum')
# print(devices)
df['IMT'] =  np.where(df['Message ID']== 0, np.nan, df['Send Time']- df['Send Time'].shift(1))


df['Receive Time'] = pd.to_datetime(df['Receive Time'], unit='ms')
df['Send Time'] = pd.to_datetime(df['Send Time'], unit='ms')
df['Latency'] = (df['Receive Time'] - df['Send Time']).dt.total_seconds() * 1000
# df['Latency'] = (df['Latency'].dt.total_seconds() * 1000)
display(df)

NameError: name 'FOLDER' is not defined

In [ ]:
!cat {FOLDER}parameters

In [ ]:
#Count, number of messages - first message in every block

df['IMT'].describe()

In [ ]:
df.groupby(['Status']).count()

In [ ]:
df['count'] = df.groupby('deviceNum')['deviceNum'].transform('count')
df['count'].describe()

In [ ]:
#get rid of the connection message
df.drop(df[df['Message ID'] < 10].index, inplace=True)
df['IMT'].describe()
# df.count()

In [ ]:
# fig = px.line(df, x="Message ID", y="Latency", color="deviceNum" ,title='Latency', render_mode='webgl')
# fig.update_traces(line=dict(width=1))
# fig.update_layout(showlegend=False)
# fig.show()


In [ ]:
df['Latency'].describe()

In [ ]:
totalMessages = df.count()['count']
freq = df.groupby(['Latency']).count()['count']  # the count of each latency group
rel = freq / totalMessages # Divide that by the total messages to get the relative count
cumsum = rel.cumsum() #Cumulative to get the cumualive frequencies
fig_cfd = go.Figure()
fig_cfd.add_trace(go.Scatter(x=cumsum.index.values, y=cumsum,mode='lines',name='lines'))
fig_cfd.update_layout(
    title = "CFD of Latency (ms)",
    xaxis_title = "Latency (ms)",
    yaxis_title = "Probability"
)
fig_cfd.show()

In [ ]:
fig_ts = px.scatter(df, x="Receive Time", y="Latency", color="deviceNum", title='Time Received vs Latency (ms) Colored by Device', render_mode='webgl')
fig_ts.update_traces(line=dict(width=1))
fig_ts.update_layout(
    showlegend=False,
    yaxis_title="Latency (ms)")
fig_ts.show()

In [ ]:
hist = px.histogram(df, x="Receive Time", y="Latency", histfunc='avg', title="Histogram of Average latencies")
hist.show()

In [ ]:
by_status_fig = px.scatter(df, x="Receive Time", y="Latency", color="Status", title='Time Received vs Latency (ms) Colored by Status')
# fig_ts = px.line(df, x="Receive Time", y="Latency", color="deviceNum", title='Time Received vs Latency (ms) Colored by Status', render_mode='webgl')
# fig_ts.update_traces(line=dict(width=1))
by_status_fig.update_layout(
    showlegend=False,
    yaxis_title="Latency (ms)")
by_status_fig.show()